<a href="https://colab.research.google.com/github/ngohonggiang/Echocardiography/blob/main/Ph%C3%A2n_lo%E1%BA%A1i_%E1%BA%A3nh_si%C3%AAu_%C3%A2m_tim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import cv2
import numpy as np
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import namedtuple
from sklearn.metrics import classification_report

In [ ]:
!git clone https://github.com/ngohonggiang/Echocardiography.git

Cloning into 'Echocardiography'...
remote: Enumerating objects: 8335, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 8335 (delta 2), reused 7 (delta 2), pack-reused 8328
Receiving objects: 100% (8335/8335), 488.02 MiB | 30.63 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Checking out files: 100% (8328/8328), done.


In [ ]:
TrainTest = namedtuple('TrainTest', ['train', 'test'])

def get_classes():
  classes = ['2C', '3C', '4C']
  return classes

def prepare_data():
  size = 224
  # raw
  transform_train_raw = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.ToTensor()
  ])

  # preprocess
  transform_train_preprocess = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.GaussianBlur(3),
    transforms.RandomRotation(2.8),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.ToTensor()
  ])

  # augmentation
  transform_train_augmentation = transforms.Compose([
    transforms.Resize((size, size)),
    transforms.RandomCrop(size, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.3, contrast=0.1),
    transforms.ToTensor()
  ])

  # test
  transform_test = transforms.Compose([
    transforms.Resize((size, size)),                                 
    transforms.ToTensor()
  ])

  trainset = torchvision.datasets.ImageFolder(
      root='/content/Echocardiography/DATA_CHAMBER_2021/train', transform=transform_train_augmentation)
  testset = torchvision.datasets.ImageFolder(
      root='/content/Echocardiography/DATA_CHAMBER_2021/test', transform=transform_test)
  return TrainTest(train=trainset, test=testset)

def prepare_loader(datasets):
  batch_size = 32
  trainloader = DataLoader(dataset=datasets.train, batch_size=batch_size, shuffle=True, num_workers=2)
  testloader = DataLoader(dataset=datasets.test, batch_size=batch_size, shuffle=False, num_workers=2)
  return TrainTest(train=trainloader, test=testloader)


In [ ]:
def train_epoch(epoch, model, loader, loss_func, optimizer, device):
  model.train()
  running_loss = 0.0
  reporting_step = 40
  for i, (images, labels) in enumerate(loader):
    images, labels = images.to(device), labels.to(device)
    
    outputs = model(images)
    loss = loss_func(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        
    running_loss += loss.item()
    if i % reporting_step == reporting_step - 1:
        print(f"Epoch {epoch} Step {i} ave_loss {running_loss/reporting_step:0.4f}")
        running_loss = 0.0

def test_epoch(epoch, model, loader, device):
  pred = []
  true = []
  with torch.no_grad():
    model.eval()
    for i, (images, labels) in enumerate(loader):
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)
      _, predicted = torch.max(outputs, dim=1)
      pred += list(predicted.cpu().numpy())
      true += list(labels.cpu().numpy())
  return pred, true

In [ ]:
def main(model=None):
  classes = get_classes()
  datasets = prepare_data()
  loaders = prepare_loader(datasets)
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  print(device)

  if model == 'vgg16':
    PATH = './vgg16.pth'
    model = torchvision.models.vgg16()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  elif model == 'vgg19':
    PATH = './vgg19.pth'
    model = torchvision.models.vgg19()
    model.classifier[6] = torch.nn.modules.Linear(in_features=4096, out_features=3)
  elif model == 'resnet50':
    PATH = './resnet50.pth'
    model = torchvision.models.resnet50()
    model.fc = torch.nn.modules.Linear(in_features=2048, out_features=3)

  model.to(device)
  loss_function = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

  for epoch in range(10):
    train_epoch(epoch, model, loaders.train, loss_function, optimizer, device)
    pred, true = test_epoch(epoch, model, loaders.test, device)
    print(classification_report(true, pred, target_names=classes))  
    print("---------------------------------------------------------------")      

    torch.save(model.state_dict(), PATH)
  return model

In [ ]:
modelVGG16 = main(model='vgg16')

In [ ]:
modelVGG19 = main(model='vgg19')

In [ ]:
modelResNet50 = main(model='resnet50')